In [7]:
import os
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input
from datetime import datetime
import numpy as np

In [2]:
def reset_random_seeds():
    '''
    Sets all necessary seed for reproduceability.
    '''
    os.environ['PYTHONHASHSEED']=str(1)
    tf.random.set_seed(1)
    np.random.seed(1)
    
reset_random_seeds()

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
BITBOARD_DIR = '../data/bitboard_autoencoder.npy'
NUM_EPOCHS = 200
LR = 0.005
LR_MULTI = 0.98
BATCH_SIZE = 75000

model_hyperparams = {
    1:{'layer1':773, 'layer2':600, 'layer3':773},
    2:{'layer1':600, 'layer2':400, 'layer3':600},
    3:{'layer1':400, 'layer2':200, 'layer3':400},
    4:{'layer1':200, 'layer2':100, 'layer3':200},
}

In [5]:
def scheduler(epoch, lr):
    return lr*0.98
reduce_lr = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [6]:
data = np.load('../data/bitboard_autoencoder_uint8.npy')

# New Pos2Vec (No Freeze Decoder)
* Layer by layer training
* Adam optimizer
* Binary cross entropy loss
* Do not freeze decoder layers

In [10]:
def create_base_autoencoder(layer1, layer2, layer3):
    model = Sequential([
        Dense(layer1, activation='relu'),
        Dense(layer2, activation='relu'),
        Dense(layer3, activation='relu')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss='binary_crossentropy')

    return model

In [11]:
def train(model, i):
    # Train it
    for epoch in range(40):
        if epoch%10 == 0:
            print(f"========================= Epoch {epoch} =========================")

        # Print time, model name and then train model.            
        print(f'AE-{i} {datetime.now().strftime("%d/%m/%Y %H:%M:%S")}, ', end='')
        model.fit(x=data, y=data, verbose=0, callbacks=[reduce_lr], batch_size=BATCH_SIZE)
        print(f'{datetime.now().strftime("%d/%m/%Y %H:%M:%S")}: {model.history.history}')

In [12]:
# Base model + Compilation
model = create_base_autoencoder(773, 600, 773)
# Train base model
train(model, 1)

# Work on adding new AEs and training accordingly:
for i in range(2, 5):
    # Remove output/decoding layer(s)
    output_layer = model.layers[-(i-1):]
    model = Sequential(model.layers[:-(i-1)])

    # Freeze weights
    for layer in model.layers:
        layer.trainable = False
    
    # Add new auto encoder to be trained
    model.add(Dense(model_hyperparams[i]['layer1'], activation='relu'))
    model.add(Dense(model_hyperparams[i]['layer2'], activation='relu'))
    model.add(Dense(model_hyperparams[i]['layer3'], activation='relu'))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss='binary_crossentropy')

    # Add previously removed output/decoding layer(s)
    for layer in output_layer:
        model.add(layer)

    # Train the model
    train(model, i)


========================= Epoch 0 =========================
AE-1 25/10/2021 00:11:30, 25/10/2021 00:11:42: {'loss': [0.11496090888977051], 'lr': [0.0049]}
AE-1 25/10/2021 00:11:42, 25/10/2021 00:11:53: {'loss': [0.08563074469566345], 'lr': [0.0048020002]}
AE-1 25/10/2021 00:11:53, 25/10/2021 00:12:04: {'loss': [0.07076152414083481], 'lr': [0.0047059604]}
AE-1 25/10/2021 00:12:04, 25/10/2021 00:12:16: {'loss': [0.06995107233524323], 'lr': [0.0046118414]}
AE-1 25/10/2021 00:12:16, 25/10/2021 00:12:27: {'loss': [0.06429088115692139], 'lr': [0.0045196046]}
AE-1 25/10/2021 00:12:27, 25/10/2021 00:12:38: {'loss': [0.06503165513277054], 'lr': [0.0044292123]}
AE-1 25/10/2021 00:12:38, 25/10/2021 00:12:50: {'loss': [0.06654562801122665], 'lr': [0.004340628]}
AE-1 25/10/2021 00:12:50, 25/10/2021 00:13:01: {'loss': [0.06455063074827194], 'lr': [0.0042538154]}
AE-1 25/10/2021 00:13:01, 25/10/2021 00:13:12: {'loss': [0.061731863766908646], 'lr': [0.004168739]}
AE-1 25/10/2021 00:13:12, 25/10/2021 0

KeyboardInterrupt: 

# New Pos2Vec (Freeze Decoder)
* Layer by layer training
* Adam optimizer
* Binary cross entropy loss
* Freeze decoder layers

In [16]:
def create_base_autoencoder(layer1, layer2, layer3):
    model = Sequential([
        Dense(layer1, activation='relu'),
        Dense(layer2, activation='relu'),
        Dense(layer3, activation='relu')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss='binary_crossentropy')

    return model

In [19]:
def train(model, i):
    # Train it
    for epoch in range(40):
        if epoch%10 == 0:
            print(f"========================= Epoch {epoch} =========================")

        # Print time, model name and then train model.            
        print(f'AE-{i} {datetime.now().strftime("%d/%m/%Y %H:%M:%S")}, ', end='')
        model.fit(x=data, y=data, verbose=0, callbacks=[reduce_lr], batch_size=BATCH_SIZE)
        print(f'{datetime.now().strftime("%d/%m/%Y %H:%M:%S")}: {model.history.history}')

In [20]:
# Base model + Compilation
fmodel = create_base_autoencoder(773, 600, 773)
# Train base model
train(fmodel, 1)

# Work on adding new AEs and training accordingly:
for i in range(2, 5):
    # Remove output/decoding layer(s)
    output_layer = fmodel.layers[-(i-1):]
    fmodel = Sequential(fmodel.layers[:-(i-1)])

    # Freeze weights
    for layer in fmodel.layers:
        layer.trainable = False
    
    # Add new auto encoder to be trained
    fmodel.add(Dense(model_hyperparams[i]['layer1'], activation='relu'))
    fmodel.add(Dense(model_hyperparams[i]['layer2'], activation='relu'))
    fmodel.add(Dense(model_hyperparams[i]['layer3'], activation='relu'))
    fmodel.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss='binary_crossentropy')

    # Add previously removed output/decoding layer(s)
    # Freeze decoder layers.
    for layer in output_layer:
        layer.trainable = False
        fmodel.add(layer)

    # Train the model
    train(fmodel, i)


========================= Epoch 0 =========================
AE-1 25/10/2021 00:45:49, 25/10/2021 00:46:02: {'loss': [0.11927720159292221], 'lr': [0.0049]}
AE-1 25/10/2021 00:46:02, 25/10/2021 00:46:14: {'loss': [0.0872572511434555], 'lr': [0.0048020002]}
AE-1 25/10/2021 00:46:14, 25/10/2021 00:46:27: {'loss': [0.07268077880144119], 'lr': [0.0047059604]}
AE-1 25/10/2021 00:46:27, 25/10/2021 00:46:39: {'loss': [0.07379947602748871], 'lr': [0.0046118414]}
AE-1 25/10/2021 00:46:39, 25/10/2021 00:46:51: {'loss': [0.07292455434799194], 'lr': [0.0045196046]}
AE-1 25/10/2021 00:46:51, 25/10/2021 00:47:04: {'loss': [0.07133720070123672], 'lr': [0.0044292123]}
AE-1 25/10/2021 00:47:04, 25/10/2021 00:47:16: {'loss': [0.07796909660100937], 'lr': [0.004340628]}
AE-1 25/10/2021 00:47:16, 25/10/2021 00:47:28: {'loss': [0.07411342114210129], 'lr': [0.0042538154]}
AE-1 25/10/2021 00:47:28, 25/10/2021 00:47:40: {'loss': [0.07224911451339722], 'lr': [0.004168739]}
AE-1 25/10/2021 00:47:40, 25/10/2021 00:

In [28]:
fpos2vec = Sequential([fmodel.layers[0], fmodel.layers[1], fmodel.layers[3], fmodel.layers[5], fmodel.layers[7]])
fpos2vec.build((None,773))

In [29]:
# Save entire model
fmodel.save_weights('../models/fautoencoders_v1/fautoencoders_v1')

# Save Pos2Vec
fpos2vec.save_weights('../models/fpos2vec_v1/fpos2vec_v1')

# Old Pos2Vec

* Layer by layer training
* SGD optimizer
* MSE loss

In [8]:
def create_base_autoencoder(layer1, layer2, layer3):
    model = Sequential([
        Dense(layer1, activation='relu'),
        Dense(layer2, activation='relu'),
        Dense(layer3, activation='relu')
    ])

    model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.005), loss='mse')

    return model

In [9]:
def train(model, i):
    # Train it
    for epoch in range(NUM_EPOCHS):
        if epoch%10 == 0:
            print(f"========================= Epoch {epoch} =========================")

        # Print time, model name and then train model.            
        print(f'AE-{i} {datetime.now().strftime("%d/%m/%Y %H:%M:%S")}, ', end='')
        model.fit(x=data, y=data, verbose=0, callbacks=[reduce_lr], batch_size=BATCH_SIZE)
        print(f'{datetime.now().strftime("%d/%m/%Y %H:%M:%S")}: {model.history.history}')

In [9]:
# Base model + Compilation
model = create_base_autoencoder(773, 600, 773)
# Train base model
train(model, 1)

# Work on adding new AEs and training accordingly:
for i in range(2, 5):
    # Remove output/decoding layer(s)
    output_layer = model.layers[-(i-1):]
    model = Sequential(model.layers[:-(i-1)])

    # Freeze weights
    for layer in model.layers:
        layer.trainable = False
    
    # Add new auto encoder to be trained
    model.add(Dense(model_hyperparams[i]['layer1'], activation='relu'))
    model.add(Dense(model_hyperparams[i]['layer2'], activation='relu'))
    model.add(Dense(model_hyperparams[i]['layer3'], activation='relu'))
    model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.005), loss='mse')

    # Add previously removed output/decoding layer(s)
    for layer in output_layer:
        model.add(layer)

    # Train the model
    train(model, i)


========================= Epoch 0 =========================
AE-1 23/10/2021 05:01:04, 23/10/2021 05:01:14: {'loss': [0.030377980321645737], 'lr': [0.0049]}
AE-1 23/10/2021 05:01:14, 23/10/2021 05:01:23: {'loss': [0.03035731613636017], 'lr': [0.0048020002]}
AE-1 23/10/2021 05:01:23, 23/10/2021 05:01:32: {'loss': [0.030337195843458176], 'lr': [0.0047059604]}
AE-1 23/10/2021 05:01:32, 23/10/2021 05:01:42: {'loss': [0.030317597091197968], 'lr': [0.0046118414]}
AE-1 23/10/2021 05:01:42, 23/10/2021 05:01:51: {'loss': [0.030298510566353798], 'lr': [0.0045196046]}
AE-1 23/10/2021 05:01:51, 23/10/2021 05:02:01: {'loss': [0.030279912054538727], 'lr': [0.0044292123]}
AE-1 23/10/2021 05:02:01, 23/10/2021 05:02:10: {'loss': [0.030261794105172157], 'lr': [0.004340628]}
AE-1 23/10/2021 05:02:10, 23/10/2021 05:02:19: {'loss': [0.030244136229157448], 'lr': [0.0042538154]}
AE-1 23/10/2021 05:02:19, 23/10/2021 05:02:29: {'loss': [0.030226917937397957], 'lr': [0.004168739]}
AE-1 23/10/2021 05:02:29, 23/10

In [10]:
pos2vec = Sequential([model.layers[0], model.layers[1], model.layers[3], model.layers[5], model.layers[7]])
pos2vec.build((None,773))

# Unfreeze weights of pos2vec
for l in pos2vec.layers:
    l.trainable = True

In [11]:
# Save entire model
model.save_weights('../models/autoencoders/autoencoders')

# Save Pos2Vec
pos2vec.save_weights('../models/pos2vec/pos2vec')

# Train Pos2Vec but freeze decoder layers each time we stack an autoencoder to be trained.

In [12]:
# Base model + Compilation
fmodel = create_base_autoencoder(773, 600, 773)
# Train base model
train(fmodel, 1)

# Work on adding new AEs and training accordingly:
for i in range(2, 5):
    # Remove output/decoding layer(s)
    output_layer = fmodel.layers[-(i-1):]
    fmodel = Sequential(fmodel.layers[:-(i-1)])

    # Freeze weights
    for layer in fmodel.layers:
        layer.trainable = False
    
    # Add new auto encoder to be trained
    fmodel.add(Dense(model_hyperparams[i]['layer1'], activation='relu'))
    fmodel.add(Dense(model_hyperparams[i]['layer2'], activation='relu'))
    fmodel.add(Dense(model_hyperparams[i]['layer3'], activation='relu'))
    fmodel.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.005), loss='mse')

    # Add previously removed output/decoding layer(s)
    # Freeze decoder layers.
    for layer in output_layer:
        layer.trainable = False
        fmodel.add(layer)

    # Train the model
    train(fmodel, i)


========================= Epoch 0 =========================
AE-1 23/10/2021 07:17:03, 23/10/2021 07:17:13: {'loss': [0.029868511483073235], 'lr': [0.0049]}
AE-1 23/10/2021 07:17:13, 23/10/2021 07:17:22: {'loss': [0.02985035814344883], 'lr': [0.0048020002]}
AE-1 23/10/2021 07:17:22, 23/10/2021 07:17:31: {'loss': [0.029832683503627777], 'lr': [0.0047059604]}
AE-1 23/10/2021 07:17:31, 23/10/2021 07:17:41: {'loss': [0.029815472662448883], 'lr': [0.0046118414]}
AE-1 23/10/2021 07:17:41, 23/10/2021 07:17:50: {'loss': [0.029798705130815506], 'lr': [0.0045196046]}
AE-1 23/10/2021 07:17:50, 23/10/2021 07:17:59: {'loss': [0.029782380908727646], 'lr': [0.0044292123]}
AE-1 23/10/2021 07:17:59, 23/10/2021 07:18:09: {'loss': [0.029766472056508064], 'lr': [0.004340628]}
AE-1 23/10/2021 07:18:09, 23/10/2021 07:18:18: {'loss': [0.029750974848866463], 'lr': [0.0042538154]}
AE-1 23/10/2021 07:18:18, 23/10/2021 07:18:27: {'loss': [0.0297358687967062], 'lr': [0.004168739]}
AE-1 23/10/2021 07:18:27, 23/10/2

In [13]:
fpos2vec = Sequential([fmodel.layers[0], fmodel.layers[1], fmodel.layers[3], fmodel.layers[5], fmodel.layers[7]])
fpos2vec.build((None,773))

# Unfreeze weights of pos2vec
for l in fpos2vec.layers:
    l.trainable = True

In [14]:
# Save entire model
fmodel.save_weights('../models/fautoencoders/fautoencoders')

# Save Pos2Vec
fpos2vec.save_weights('../models/fpos2vec/fpos2vec')